# Transform pushing

In [1]:
import os
import sys
sys.path += ['../build', '../../../build', '../../lib']
# print(sys.version)
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}
** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'


** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}



## Generating World

Set some variables.

In [2]:
def get_world(folder, idx):
    world_list = []

    file = f'{idx}.txt'

    f = open(folder+file, 'r')
    
    for line in f:
        row = []

        for char in line:
            if char == '\n':
                break

            row.append([int(char)])

        world_list.append(row)


    world = np.asarray(world_list)
#     world = world[1:-1,1:-1]
    off = -np.floor(world.shape[0] / 2) + 0.5
    
    return world, off

Iterate over world Array and generate Environment.

In [7]:
def transform_world(C, world, off, colors):
    pusher = C.addFile('/home/basti/Dokumente/Bachelorarbeit/rai-python/my-robotModels/cylRobo/robo.g')

    for x in range(world.shape[0]):
        for y in range(world.shape[1]):
            if(world[x][y] == 5):
                continue
            
            empty = C.addFrame(name=f'empty_{x}_{y}', parent='floor', args='type:ssBox size:[1, 1, 1, 0], contact, joint:rigid')
            empty.setPosition([(x+off), (y+off), -0.5])
            empty.setMass(100.0)
            
            if(world[x][y] == 1):
                c = colors[(x+y) % 2]
                block = C.addFrame(name=f'block_{x}_{y}', parent='floor', args=f'type:ssBox size:[1, 1, 1, .02] color=[{c[0]}, {c[1]}, {c[2]}], contact, joint:rigid')
                block.setPosition([(x+off), (y+off), 0.5])
                block.setMass(100.0)

            if(world[x][y] == 2):
                pusher = C.getFrame(frameName='robo')
                pusher.setPosition([(x+off), (y+off), 0.1])

            if(world[x][y] == 3):
                box = C.addFrame(name=f'box_{x}_{y}', parent='floor', args='type:ssBox size:[.9, .9, 1, .02] color=[.6, .3, 0.], contact')
                box.setPosition([(x+off), (y+off), 0.5])
                box.setMass(100.0)
            if(world[x][y] == 4):
                goal = C.addFrame(name=f'goal_{x}_{y}', parent='floor', args='type:ssBox size=[.95, .95, .002, .002] color:[0., 1., 0.], joint:rigid')
                goal.setPosition([(x+off), (y+off), -0.001])


In [8]:
run = 1

########
SAFE_DIR = f'/home/basti/Dokumente/Bachelorarbeit/gym-pcgrl/shared_runs/pushing_turtle_{run}_log/generated/'
colors = [[.8, .1, .1], [1., .3, .2]]

n = -1
listdir = os.listdir(SAFE_DIR)
for f in listdir:
    try:
        val = int(f.split('.')[0])
        n = max(n, val)
    except ValueError:
        continue      
    
for i in range(0, n+1):
    C = ry.Config()
    C.clear()
        
    world, off = get_world(SAFE_DIR, i)
    
#     floor = C.addFrame(name='floor', args=f'type:ssBox size=[{world.shape[0]}, {world.shape[1]}, .01, 0] ')
#     floor.setPosition([0, 0, -0.005])
        
    transform_world(C, world, off, colors)
        
    g_file = f'{i}.g'
    C.save(SAFE_DIR+g_file)
    C.view()

In [7]:
C = ry.Config()
C.clear()

run = 1
file = 0
C.addFile(f'/home/basti/Dokumente/Bachelorarbeit/gym-pcgrl/shared_runs/pushing_turtle_{run}_log/generated/{file}.g')
# C.addFile('../my-robotModels/p30-puzzle5.g')
C.view()

In [ ]:
pusher = C.addFrame(name=f'pusher', parent='floor', args=f'type:cylinder size:[0.02, 0.03] color=[0.6, 1, 0.5]')
pusher.setPosition([(x+off), (y+off), 0.11])